[View in Colaboratory](https://colab.research.google.com/github/gauthamk28/mnist-custom-digit-prediction/blob/master/Gautham_Batch_2_Assignment_4B(87_77).ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 15
num_filter = 18
compression = 1
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

 75472896/170498071 [============>.................] - ETA: 1:18

170500096/170498071 [==============================] - 97s 1us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 18, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 18, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 18
dropout_rate = 0.2
l = 15
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 18)   486         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 18)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (


batch_normalization_55 (BatchNo (None, 4, 4, 126)    504         concatenate_51[0][0]             
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 4, 4, 126)    0           batch_normalization_55[0][0]     
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 4, 4, 18)     20412       activation_55[0][0]              
__________________________________________________________________________________________________
dropout_55 (Dropout)            (None, 4, 4, 18)     0           conv2d_56[0][0]                  
__________________________________________________________________________________________________
concatenate_52 (Concatenate)    (None, 4, 4, 144)    0           concatenate_51[0][0]             
                                                                 dropout_55[0][0]                 
_________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
import os
os.mkdir('save_files')
os.chdir('save_files')
!ls

In [0]:
!ls

In [0]:
#Creating Checkpoints after every epoch

from keras.callbacks import ModelCheckpoint

filepath="Final_may15-weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=True,
                             mode='max', period=1)
callbacks_list = [checkpoint]

In [0]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks_list,validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 466s 9ms/step - loss: 1.4807 - acc: 0.4554 - val_loss: 1.5927 - val_acc: 0.4670

Epoch 00001: val_acc improved from -inf to 0.46700, saving model to Final_may15-weights-01-0.47.hdf5
Epoch 2/50
13952/50000 [=======>......................] - ETA: 4:59 - loss: 1.1120 - acc: 0.5954

50000/50000 [==============================] - 444s 9ms/step - loss: 1.0252 - acc: 0.6317 - val_loss: 1.4497 - val_acc: 0.5518

Epoch 00002: val_acc improved from 0.46700 to 0.55180, saving model to Final_may15-weights-02-0.55.hdf5
Epoch 3/50
29440/50000 [================>.............] - ETA: 2:50 - loss: 0.8604 - acc: 0.6944

50000/50000 [==============================] - 443s 9ms/step - loss: 0.8359 - acc: 0.7032 - val_loss: 1.2236 - val_acc: 0.6156

Epoch 00003: val_acc improved from 0.55180 to 0.61560, saving model to Final_may15-weights-03-0.62.hdf5
Epoch 4/50
33408/50000 [===================>..........] - ETA: 2:17 - loss: 0.7245 - acc: 0.7424

50000/50000 [==============================] - 444s 9ms/step - loss: 0.7166 - acc: 0.7450 - val_loss: 1.4535 - val_acc: 0.6027

Epoch 00004: val_acc did not improve from 0.61560
Epoch 5/50
41344/50000 [=======================>......] - ETA: 1:11 - loss: 0.6543 - acc: 0.7696

50000/50000 [==============================] - 444s 9ms/step - loss: 0.6559 - acc: 0.7691 - val_loss: 0.8771 - val_acc: 0.7202

Epoch 00005: val_acc improved from 0.61560 to 0.72020, saving model to Final_may15-weights-05-0.72.hdf5
Epoch 6/50
36480/50000 [====================>.........] - ETA: 1:52 - loss: 0.5968 - acc: 0.7899

...

50000/50000 [==============================] - 444s 9ms/step - loss: 0.5406 - acc: 0.8101 - val_loss: 0.7048 - val_acc: 0.7783

Epoch 00007: val_acc improved from 0.73350 to 0.77830, saving model to Final_may15-weights-07-0.78.hdf5
Epoch 8/50
34944/50000 [===================>..........] - ETA: 2:05 - loss: 0.4960 - acc: 0.8269

50000/50000 [==============================] - 444s 9ms/step - loss: 0.5002 - acc: 0.8245 - val_loss: 0.7610 - val_acc: 0.7647

Epoch 00008: val_acc did not improve from 0.77830
Epoch 9/50
41856/50000 [========================>.....] - ETA: 1:07 - loss: 0.4667 - acc: 0.8372

50000/50000 [==============================] - 444s 9ms/step - loss: 0.4688 - acc: 0.8367 - val_loss: 1.0880 - val_acc: 0.7020

Epoch 00009: val_acc did not improve from 0.77830
Epoch 10/50
43776/50000 [=========================>....] - ETA: 51s - loss: 0.4378 - acc: 0.8478

50000/50000 [==============================] - 443s 9ms/step - loss: 0.4382 - acc: 0.8475 - val_loss: 0.8505 - val_acc: 0.7495

Epoch 00010: val_acc did not improve from 0.77830
Epoch 11/50
44416/50000 [=========================>....] - ETA: 46s - loss: 0.4100 - acc: 0.8581

50000/50000 [==============================] - 443s 9ms/step - loss: 0.4106 - acc: 0.8578 - val_loss: 0.8843 - val_acc: 0.7566

Epoch 00011: val_acc did not improve from 0.77830
Epoch 12/50
14976/50000 [=======>......................] - ETA: 4:50 - loss: 0.3802 - acc: 0.8653

In [16]:
!ls

Final_may15-weights-01-0.47.hdf5  Final_may15-weights-05-0.72.hdf5
Final_may15-weights-02-0.55.hdf5  Final_may15-weights-06-0.73.hdf5
Final_may15-weights-03-0.62.hdf5  Final_may15-weights-07-0.78.hdf5


In [0]:
model.save('Final_may15-weights-07-0.78.hdf5')

from keras.models import load_model
model=load_model('Final_may15-weights-07-0.78.hdf5')

In [0]:
#Creating Checkpoints after every epoch

from keras.callbacks import ModelCheckpoint

filepath="Final_from8-weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=True,
                             mode='max', period=1)
callbacks_list = [checkpoint]

In [19]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=43,verbose=1,callbacks=callbacks_list,validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/43
50000/50000 [==============================] - 455s 9ms/step - loss: 0.3696 - acc: 0.8700 - val_loss: 0.8770 - val_acc: 0.7500

Epoch 00001: val_acc improved from -inf to 0.75000, saving model to Final_from8-weights-01-0.75.hdf5
Epoch 2/43
13952/50000 [=======>......................] - ETA: 4:59 - loss: 0.3380 - acc: 0.8836

50000/50000 [==============================] - 444s 9ms/step - loss: 0.3547 - acc: 0.8770 - val_loss: 0.7606 - val_acc: 0.7885

Epoch 00002: val_acc improved from 0.75000 to 0.78850, saving model to Final_from8-weights-02-0.79.hdf5
Epoch 3/43
29440/50000 [================>.............] - ETA: 2:51 - loss: 0.3239 - acc: 0.8874

50000/50000 [==============================] - 445s 9ms/step - loss: 0.3347 - acc: 0.8837 - val_loss: 0.5844 - val_acc: 0.8216

Epoch 00003: val_acc improved from 0.78850 to 0.82160, saving model to Final_from8-weights-03-0.82.hdf5
Epoch 4/43
33408/50000 [===================>..........] - ETA: 2:18 - loss: 0.3142 - acc: 0.8885

50000/50000 [==============================] - 445s 9ms/step - loss: 0.3180 - acc: 0.8881 - val_loss: 0.5763 - val_acc: 0.8358

Epoch 00004: val_acc improved from 0.82160 to 0.83580, saving model to Final_from8-weights-04-0.84.hdf5
Epoch 5/43
34432/50000 [===================>..........] - ETA: 2:09 - loss: 0.3012 - acc: 0.8934

50000/50000 [==============================] - 445s 9ms/step - loss: 0.3064 - acc: 0.8918 - val_loss: 0.7681 - val_acc: 0.7938

Epoch 00005: val_acc did not improve from 0.83580
Epoch 6/43
41728/50000 [========================>.....] - ETA: 1:08 - loss: 0.2882 - acc: 0.8964

...

50000/50000 [==============================] - 445s 9ms/step - loss: 0.0883 - acc: 0.9683 - val_loss: 0.7060 - val_acc: 0.8583

Epoch 00039: val_acc did not improve from 0.86100
Epoch 40/43
44672/50000 [=========================>....] - ETA: 44s - loss: 0.0839 - acc: 0.9699

50000/50000 [==============================] - 445s 9ms/step - loss: 0.0847 - acc: 0.9697 - val_loss: 0.7178 - val_acc: 0.8526

Epoch 00040: val_acc did not improve from 0.86100
Epoch 41/43
44672/50000 [=========================>....] - ETA: 44s - loss: 0.0841 - acc: 0.9706

50000/50000 [==============================] - 445s 9ms/step - loss: 0.0863 - acc: 0.9700 - val_loss: 0.5716 - val_acc: 0.8777

Epoch 00041: val_acc improved from 0.86100 to 0.87770, saving model to Final_from8-weights-41-0.88.hdf5
Epoch 42/43
37248/50000 [=====================>........] - ETA: 1:46 - loss: 0.0864 - acc: 0.9688

50000/50000 [==============================] - 445s 9ms/step - loss: 0.0860 - acc: 0.9687 - val_loss: 0.6041 - val_acc: 0.8770

Epoch 00042: val_acc did not improve from 0.87770
Epoch 43/43
42368/50000 [========================>.....] - ETA: 1:03 - loss: 0.0789 - acc: 0.9719

50000/50000 [==============================] - 445s 9ms/step - loss: 0.0799 - acc: 0.9716 - val_loss: 0.6254 - val_acc: 0.8749

Epoch 00043: val_acc did not improve from 0.87770


In [22]:
model.load_weights("Final_from8-weights-41-0.88.hdf5")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 18)   486         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 18)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [23]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 37s 4ms/step
Test loss: 0.5716400339663029
Test accuracy: 0.8777


In [25]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_2.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_2.h5')

Since I use FireFox browser , i used the below method to download the model to disk(to google drive and then disk)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [30]:
uploaded = drive.CreateFile({'model': 'Final_from8-weights-41-0.88.hdf5'})
uploaded.SetContentFile('Final_from8-weights-41-0.88.hdf5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1awizp9ecRCa-zSuhgHFjTIwgYHJ7mS0J


In [31]:
uploaded_weights = drive.CreateFile({'model_weights': 'DNST_model_2.h5'})
uploaded_weights.SetContentFile('DNST_model_2.h5')
uploaded_weights.Upload()
print('Uploaded file with ID {}'.format(uploaded_weights.get('id')))

Uploaded file with ID 1mGGWWTqOS1vtSCi6j05fLBrMdZV7HZ2w
